In [ ]:
#Daniel Canton (2024013958), , Gustavo José Vieira (2024014164), Thiago Henrique Silva de Almeida (2024014180).

import sqlite3
import pandas as pd # Importando pandas para melhor visualização

# Nome do arquivo do banco de dados (será criado se não existir)
db_file = 'cineufmg.db'

try:
    # Conectar ao banco de dados
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Comandos SQL de criação e população
    sql_creation_and_population = """
    -- Habilitar a verificação de chaves estrangeiras (importante para SQLite)
    PRAGMA foreign_keys = ON;

    -- Remover tabelas existentes para garantir um ambiente limpo (opcional, útil para testes)
    DROP TABLE IF EXISTS ingressos;
    DROP TABLE IF EXISTS sessoes;
    DROP TABLE IF EXISTS clientes;
    DROP TABLE IF EXISTS filmes;
    DROP TABLE IF EXISTS salas;

    -- 1. Criação da tabela 'filmes'
    CREATE TABLE filmes (
        id INTEGER PRIMARY KEY,
        titulo TEXT NOT NULL,
        genero TEXT NOT NULL,
        duracao INTEGER NOT NULL, -- Duração em minutos
        classificacao_indicativa INTEGER -- Idade mínima recomendada
    );

    -- 2. Criação da tabela 'salas'
    CREATE TABLE salas (
        id INTEGER PRIMARY KEY,
        nome TEXT NOT NULL,
        capacidade INTEGER NOT NULL
    );

    -- 3. Criação da tabela 'sessoes'
    CREATE TABLE sessoes (
        id INTEGER PRIMARY KEY,
        filme_id INTEGER NOT NULL,
        sala_id INTEGER NOT NULL,
        data_hora TEXT NOT NULL, -- Formato 'YYYY-MM-DD HH:MM:SS'
        preco_ingresso REAL NOT NULL,
        FOREIGN KEY (filme_id) REFERENCES filmes(id),
        FOREIGN KEY (sala_id) REFERENCES salas(id)
    );

    -- 4. Criação da tabela 'clientes'
    CREATE TABLE clientes (
        id INTEGER PRIMARY KEY,
        nome TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        data_nascimento TEXT -- Formato 'YYYY-MM-DD'
    );

    -- 5. Criação da tabela 'ingressos'
    CREATE TABLE ingressos (
        id INTEGER PRIMARY KEY,
        sessao_id INTEGER NOT NULL,
        cliente_id INTEGER NOT NULL,
        data_compra TEXT NOT NULL, -- Formato 'YYYY-MM-DD HH:MM:SS'
        FOREIGN KEY (sessao_id) REFERENCES sessoes(id),
        FOREIGN KEY (cliente_id) REFERENCES clientes(id)
    );


    -- População das tabelas com dados de exemplo

    -- Inserção de dados na tabela 'filmes'
    INSERT INTO filmes (id, titulo, genero, duracao, classificacao_indicativa) VALUES
    (1, 'Vingadores: Ultimato', 'Ação', 181, 12),
    (2, 'O Rei Leão', 'Animação', 118, 0),
    (3, 'Parasita', 'Drama', 132, 18), -- ALTERADO: Classificação para 18
    (4, 'Minha Mãe é Uma Peça 3', 'Comédia', 111, 12),
    (5, 'Interestelar', 'Ficção Científica', 169, 10),
    (6, 'Annabelle 3: De Volta Para Casa', 'Terror', 106, 14),
    (7, 'Forrest Gump: O Contador de Histórias', 'Drama', 142, 10),
    (8, 'Central do Brasil', 'Drama', 113, 12),
    (9, 'Super Mario Bros. O Filme', 'Animação', 92, 0),
    (10, 'Oppenheimer', 'Drama', 180, 14),
    (11, 'Barbie', 'Comédia', 114, 12);

    -- Inserção de dados na tabela 'salas'
    INSERT INTO salas (id, nome, capacidade) VALUES
    (1, 'Sala 1 - IMAX', 250),
    (2, 'Sala 2 - VIP', 100),
    (3, 'Sala 3 - Padrão', 180),
    (4, 'Sala 4 - Padrão', 180),
    (5, 'Sala 5 - Pequena', 80);

    -- Inserção de dados na tabela 'clientes'
    INSERT INTO clientes (id, nome, email, data_nascimento) VALUES
    (1, 'Ana Silva', 'ana.silva@email.com', '1990-05-15'),
    (2, 'Bruno Costa', 'bruno.costa@email.com', '1988-11-20'),
    (3, 'Carla Dias', 'carla.dias@email.com', '2000-01-30'),
    (4, 'Daniel Alves', 'daniel.alves@email.com', '1995-07-22'),
    (5, 'Eduarda Lima', 'eduarda.lima@email.com', '2002-03-01'),
    (6, 'Fernando Rocha', 'fernando.rocha@email.com', '1975-09-10');


    -- Inserção de dados na tabela 'sessoes' (depende de 'filmes' e 'salas')
    INSERT INTO sessoes (id, filme_id, sala_id, data_hora, preco_ingresso) VALUES
    (1, 1, 1, '2025-04-25 10:00:00', 35.00),
    (2, 4, 3, '2025-04-25 14:30:00', 28.00),
    (3, 3, 2, '2025-04-25 19:00:00', 40.00),
    (4, 1, 1, '2025-04-25 22:00:00', 38.00),
    (5, 2, 4, '2025-04-26 11:00:00', 25.00),
    (6, 5, 1, '2025-04-26 15:00:00', 35.00),
    (7, 4, 3, '2025-04-27 18:30:00', 30.00),
    (8, 6, 5, '2025-04-27 21:00:00', 32.00),
    (9, 1, 1, '2025-04-28 10:00:00', 35.00),
    (10, 7, 3, '2025-04-28 14:00:00', 28.00),
    (11, 8, 4, '2025-04-29 16:00:00', 28.00),
    (12, 9, 3, '2025-04-30 10:30:00', 25.00),
    (13, 10, 2, '2025-04-30 19:30:00', 42.00),
    (14, 11, 4, '2025-05-01 17:00:00', 30.00),
    (15, 11, 4, '2025-05-01 20:00:00', 32.00),
    (16, 1, 3, '2025-05-02 10:00:00', 30.00); -- NOVO: 4ª sessão para Vingadores: Ultimato (filme_id 1)


    -- Inserção de dados na tabela 'ingressos' (depende de 'sessoes' e 'clientes')
    INSERT INTO ingressos (id, sessao_id, cliente_id, data_compra) VALUES
    (1, 1, 1, '2025-04-20 09:00:00'),
    (2, 1, 2, '2025-04-20 09:05:00'),
    (3, 2, 1, '2025-04-20 11:00:00'),
    (4, 3, 3, '2025-04-21 10:00:00'),
    (5, 3, 4, '2025-04-21 10:15:00'),
    (6, 4, 1, '2025-04-22 15:00:00'),
    (7, 5, 2, '2025-04-23 09:30:00'),
    (8, 5, 3, '2025-04-23 09:40:00'),
    (9, 6, 4, '2025-04-24 11:00:00'),
    (10, 7, 5, '2025-04-25 17:00:00'),
    (11, 7, 5, '2025-04-25 17:05:00'),
    (12, 7, 5, '2025-04-25 17:10:00'),
    (13, 9, 6, '2025-04-26 12:00:00'),
    (14, 10, 1, '2025-04-27 10:00:00'),
    (15, 13, 2, '2025-04-28 14:00:00'),
    (16, 14, 3, '2025-04-29 11:00:00');
    """
    
    # Executar todos os comandos SQL de criação e população
    cursor.executescript(sql_creation_and_population)
    
    # Commit (salvar) as mudanças no banco de dados
    conn.commit()
    print(f"Banco de dados '{db_file}' criado e populado com sucesso!\n")

    # --- Funções auxiliares para executar e exibir consultas ---
    def execute_query(query_name, sql_query):
        print(f"--- {query_name} ---")
        try:
            cursor.execute(sql_query)
            rows = cursor.fetchall()
            if rows:
                # Obter nomes das colunas (para DataFrame)
                col_names = [description[0] for description in cursor.description]
                df = pd.DataFrame(rows, columns=col_names)
                print(df.to_string(index=False)) # to_string para evitar truncamento em notebooks
            else:
                print("Nenhum resultado encontrado.")
        except sqlite3.Error as e:
            print(f"Erro na consulta: {e}")
        print("\n")

    # --- Especificação das Consultas ---

    # 5.1 Consultas em Álgebra Relacional

    # A1. Pi_titulo (Sigma_genero='Ação' (filmes))
    execute_query(
        "A1. Filmes do gênero 'Ação'",
        "SELECT titulo FROM filmes WHERE genero = 'Ação';"
    )

    # A2. Pi_nome (Sigma_capacidade>100 (salas))
    execute_query(
        "A2. Salas com capacidade superior a 100",
        "SELECT nome FROM salas WHERE capacidade > 100;"
    )

    # A3. Pi_titulo,preco_ingresso (sessoes JOIN filmes)
    execute_query(
        "A3. Título do filme e preço do ingresso por sessão",
        "SELECT f.titulo, s.preco_ingresso FROM sessoes s JOIN filmes f ON s.filme_id = f.id;"
    )

    # A4. Pi_nome (clientes JOIN ingressos)
    execute_query(
        "A4. Nomes de clientes que compraram ingressos",
        "SELECT DISTINCT c.nome FROM clientes c JOIN ingressos i ON c.id = i.cliente_id;"
    )

    # A5. Pi_nome (Sigma_classificacao_indicativa>16 (filmes))
    # Usando 'titulo' em vez de 'nome' conforme o esquema da tabela 'filmes'.
    execute_query(
        "A5. Filmes com classificação indicativa maior que 16",
        "SELECT titulo FROM filmes WHERE classificacao_indicativa > 16;"
    )

    # A6. Pi_nome,data_compra (clientes JOIN ingressos JOIN sessoes)
    # A consulta original da álgebra relacional sugere um join de 3 tabelas, mas a data_compra está diretamente em ingressos.
    # Assumimos que a intenção é mostrar o nome do cliente e a data da compra do ingresso.
    execute_query(
        "A6. Nome do cliente e data de compra do ingresso",
        "SELECT c.nome, i.data_compra FROM clientes c JOIN ingressos i ON c.id = i.cliente_id JOIN sessoes s ON i.sessao_id = s.id;"
    )

    # A7. Pi_titulo (filmes JOIN sessoes)
    execute_query(
        "A7. Títulos de filmes que possuem sessões programadas",
        "SELECT DISTINCT f.titulo FROM filmes f JOIN sessoes s ON f.id = s.filme_id;"
    )

    # A8. Pi_sessao_id (Sigma_data_hora LIKE '2025-04-25%' (sessoes))
    execute_query(
        "A8. IDs de sessões que ocorrem em 25/04/2025",
        "SELECT id FROM sessoes WHERE data_hora LIKE '2025-04-25%';"
    )

    # A9. cliente_id, COUNT(*) (ingressos) agrupado por cliente_id
    execute_query(
        "A9. Contagem de ingressos por cliente",
        "SELECT cliente_id, COUNT(*) AS total_ingressos FROM ingressos GROUP BY cliente_id;"
    )

    # A10. Pi_titulo (filmes JOIN sessoes JOIN ingressos)
    execute_query(
        "A10. Títulos de filmes que tiveram ingressos vendidos",
        "SELECT DISTINCT f.titulo FROM filmes f JOIN sessoes s ON f.id = s.filme_id JOIN ingressos i ON s.id = i.sessao_id;"
    )

    # 5.2 Consultas em Linguagem Natural

    # B1. Liste todos os filmes do gênero 'Comédia'.
    execute_query(
        "B1. Filmes do gênero 'Comédia'",
        "SELECT * FROM filmes WHERE genero = 'Comédia';"
    )

    # B2. Encontre as salas que possuem capacidade superior a 150 lugares.
    execute_query(
        "B2. Salas com capacidade superior a 150 lugares",
        "SELECT * FROM salas WHERE capacidade > 150;"
    )

    # B3. Liste as sessões que ocorrem no período noturno (após 18h).
    execute_query(
        "B3. Sessões que ocorrem após as 18h",
        "SELECT * FROM sessoes WHERE STRFTIME('%H', data_hora) >= '18';"
    )

    # B4. Mostre os clientes que compraram mais de 2 ingressos.
    execute_query(
        "B4. Clientes que compraram mais de 2 ingressos",
        "SELECT c.nome, COUNT(i.id) AS total_ingressos FROM clientes c JOIN ingressos i ON c.id = i.cliente_id GROUP BY c.id HAVING total_ingressos > 2;"
    )

    # B5. Calcule a média de preço dos ingressos vendidos por sessão.
    # Interpretação: A média do preco_ingresso das sessões que tiveram pelo menos um ingresso vendido.
    execute_query(
        "B5. Média de preço dos ingressos das sessões que tiveram vendas",
        "SELECT AVG(s.preco_ingresso) AS media_preco_ingressos_vendidos FROM sessoes s INNER JOIN ingressos i ON s.id = i.sessao_id;"
    )

    # B6. Liste as sessões que ainda não possuem ingressos vendidos.
    execute_query(
        "B6. Sessões sem ingressos vendidos",
        "SELECT * FROM sessoes WHERE id NOT IN (SELECT DISTINCT sessao_id FROM ingressos);"
    )

    # B7. Liste os filmes com mais de 3 sessões programadas.
    execute_query(
        "B7. Filmes com mais de 3 sessões programadas",
        "SELECT f.titulo, COUNT(s.id) AS num_sessoes FROM filmes f JOIN sessoes s ON f.id = s.filme_id GROUP BY f.id HAVING num_sessoes > 3;"
    )

    # B8. Encontre os clientes que assistiram a filmes de 'Drama'.
    execute_query(
        "B8. Clientes que assistiram a filmes de 'Drama'",
        "SELECT DISTINCT c.nome FROM clientes c JOIN ingressos i ON c.id = i.cliente_id JOIN sessoes s ON i.sessao_id = s.id JOIN filmes f ON s.filme_id = f.id WHERE f.genero = 'Drama';"
    )

    # B9. Liste as sessões do filme 'Vingadores: Ultimato'.
    execute_query(
        "B9. Sessões do filme 'Vingadores: Ultimato'",
        "SELECT s.* FROM sessoes s JOIN filmes f ON s.filme_id = f.id WHERE f.titulo = 'Vingadores: Ultimato';"
    )

    # B10. Qual o dia com maior número de ingressos vendidos?
    execute_query(
        "B10. Dia com maior número de ingressos vendidos",
        "SELECT STRFTIME('%Y-%m-%d', data_compra) AS dia_compra, COUNT(id) AS total_ingressos FROM ingressos GROUP BY dia_compra ORDER BY total_ingressos DESC LIMIT 1;"
    )

except sqlite3.Error as e:
    print(f"Erro geral: {e}")
finally:
    # Fechar a conexão
    if conn:
        conn.close()
        print("Conexão com o banco de dados fechada.")